## Instalando e Carregando os Pacotes

In [ ]:
# Imports dos pacotes
import re
import nltk
import string
import numpy as np
import pandas as pd
from os import getcwd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

## Extração e Carga dos Dados

In [ ]:
# Download dos dados
nltk.download('twitter_samples')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


True

In [ ]:
# Download das stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Verifica a pasta
getcwd()

'/content'

In [ ]:
# Importa o Corpus (dados)
from nltk.corpus import twitter_samples

In [ ]:
type(twitter_samples)

nltk.corpus.reader.twitter.TwitterCorpusReader

## Preparação dos Dados

O objeto `twitter_samples` contém subconjuntos de 5 mil tweets positivos, 5 mil tweets negativos e o conjunto completo de 10.000 tweets.

Vamos trabalhar com as duas amostras de 5 mil tweets cada uma.

In [ ]:
# Carregando os tweets positivos
tweets_positivos = twitter_samples.strings('positive_tweets.json')

In [ ]:
len(tweets_positivos)

5000

In [ ]:
# Amostra de alguns tweets positivos
tweets_positivos[2:8]

['@DespiteOfficial we had a listen last night :) As You Bleed is an amazing track. When are you in Scotland?!',
 '@97sides CONGRATS :)',
 'yeaaaah yippppy!!!  my accnt verified rqst has succeed got a blue tick mark on my fb profile :) in 15 days',
 '@BhaktisBanter @PallaviRuhail This one is irresistible :)\n#FlipkartFashionFriday http://t.co/EbZ0L2VENM',
 "We don't like to keep our lovely customers waiting for long! We hope you enjoy! Happy Friday! - LWWF :) https://t.co/smyYriipxI",
 '@Impatientraider On second thought, there’s just not enough time for a DD :) But new shorts entering system. Sheep must be buying.']

In [ ]:
# Carregando os tweets negativos
tweets_negativos = twitter_samples.strings('negative_tweets.json')

In [ ]:
len(tweets_negativos)

5000

In [ ]:
# Amostra de alguns tweets negativos
tweets_negativos[2:8]

['@Hegelbon That heart sliding into the waste basket. :(',
 '“@ketchBurning: I hate Japanese call him "bani" :( :(”\n\nMe too',
 'Dang starting next week I have "work" :(',
 "oh god, my babies' faces :( https://t.co/9fcwGvaki0",
 '@RileyMcDonough make me smile :((',
 '@f0ggstar @stuartthull work neighbour on motors. Asked why and he said hates the updates on search :( http://t.co/XvmTUikWln']

In [ ]:
print('Positivos:',type(tweets_positivos))
print('Negativos:',type(tweets_negativos))

Positivos: <class 'list'>
Negativos: <class 'list'>


## Divisão em Treino e Teste

Vamos dividir os dados com uma proporção 80/20 (treino/teste) garantindo a mesma proporção de tweets positivos e negativos.

In [ ]:
# Divisão em treino e teste(80/20), separando 4000(80%) de tweets positivos para teste e 1000(20%) para treino
tweets_positivos_teste = tweets_positivos[4000:]
tweets_positivos_treino = tweets_positivos[:4000]

In [ ]:
# Divisão em treino e teste(80/20), separando 4000(80%) de tweets negativo para teste e 1000(20%) para treino
tweets_negativos_teste = tweets_negativos[4000:]
tweets_negativos_treino = tweets_negativos[:4000]

In [ ]:
# Unindo os dataset's de treino
dados_treino_x = tweets_positivos_treino + tweets_negativos_treino

In [ ]:
# 4000 + 4000 = 8000
len(dados_treino_x)

8000

In [ ]:
# Unindo os dataset's de teste
dados_teste_x = tweets_positivos_teste + tweets_negativos_teste

In [ ]:
# 1000 + 1000 = 2000
len(dados_teste_x)

2000

In [ ]:
# Dando valor para positivos(1) e negativos(0) no dataset de treino
y_treino = np.append(np.ones((len(tweets_positivos_treino), 1)),
                     np.zeros((len(tweets_negativos_treino), 1)), axis = 0)

In [ ]:
y_treino.shape

(8000, 1)

In [ ]:
# Dando valor para positivos(1) e negativos(0) no dataset de teste
y_teste = np.append(np.ones((len(tweets_positivos_teste), 1)),
                    np.zeros((len(tweets_negativos_teste), 1)), axis = 0)

In [ ]:
y_teste.shape

(2000, 1)

## Manipulação de Texto e Pré-Processamento dos Dados

Vamos criar uma função para processar o texto dos posts do Twitter.

In [ ]:
# Função para limpeza e processamento dos tweets
def limpa_processa_tweet(tweet):

    # Retirar as stopword(artigo, pronome, ...), em ingles por que os tweets estão em ingles
    stopwords_ingles = stopwords.words('english')

    # Remover o $ em caso como $GE(colado com uma palavra), e vou substituir por vazio ''
    tweet = re.sub(r'\$\w*', '', tweet)

    # Remover o 'RT'(colado com uma palavra), e vou substituir por vazio ''
    tweet = re.sub(r'^RT[\s]+', '', tweet)

    # Remove hyperlinks, removendo o HTTP ou HTTPS e substituindo por ''(vazio)
    tweet = re.sub(r'http?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)

    # Remove hashtags e substituir por ''(vazio)
    tweet = re.sub(r'#', '', tweet)

    # Cria um tokenizador, para separar as frases em palavras
    tokenizer = TweetTokenizer(preserve_case = False, strip_handles = True, reduce_len = True)

    # Aplica o tokenizador
    tweet_tokens = tokenizer.tokenize(tweet)

    # Lista para os tweets limpos
    tweets_tratados = []

    # Cria o objeto Stemmer(função para deixar os radicais das palavras)
    stemmer = PorterStemmer()

    # Loop para percorrer as palavras na lista dos tweets
    for palavra in tweet_tokens:

        # Removendo as stop words e pontuação
        if (palavra not in stopwords_ingles and palavra not in string.punctuation):

            # Aplicando o STEMMER
            radical_palavra = stemmer.stem(palavra)

            # Tweets limpos
            tweets_tratados.append(radical_palavra)

    return tweets_tratados

In [ ]:
# Tweet original
print('Este é um exemplo de um tweet positivo original: \n\n', dados_treino_x[0])

Este é um exemplo de um tweet positivo original: 

 #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)


In [ ]:
# Tweet tratado
print('\nEste é um exemplo da versão processada do tweet: \n\n', limpa_processa_tweet(dados_treino_x[0]))


Este é um exemplo da versão processada do tweet: 

 ['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']


In [ ]:
# Função para criar o dicionário de frequência das palavras
def cria_freqs(tweets, ys):

    # tweets é uma lista de tweets
    # ys é um array m x 1 com um label de sentimento para cada tweet (0 ou 1)

    # Sqeeeze (remove uma das dimensões)
    yslist = np.squeeze(ys).tolist()

    # Dicionário de frequências
    freqs = {}

    # Loop para cada tweet
    for y, tweet in zip(yslist, tweets):

        # Loop para cada palavra
        for word in limpa_processa_tweet(tweet):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1

    return freqs

In [ ]:
# Exemplo de tweets
dados_treino_x[0:5]

['#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)',
 '@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!',
 '@DespiteOfficial we had a listen last night :) As You Bleed is an amazing track. When are you in Scotland?!',
 '@97sides CONGRATS :)',
 'yeaaaah yippppy!!!  my accnt verified rqst has succeed got a blue tick mark on my fb profile :) in 15 days']

In [ ]:
# Exemplo da categoria(positivo/negativo)
y_treino[0:5]

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [ ]:
# função de frequencia nos dados de treino
freqs = cria_freqs(dados_treino_x, y_treino)

In [ ]:
type(freqs)
# Tipo dicionario

dict

In [ ]:
#Exemplo de valores positivos             # Nesse trecho[50 - 55]:
dict(list(freqs.items())[50:55])          # Happi - Happy - Apareceu 161 vezes em tweets positivos
                                          # Friday - Apareceu 91 em tweets positivos

{('long', 1.0): 27,
 ('hope', 1.0): 113,
 ('enjoy', 1.0): 57,
 ('happi', 1.0): 161,
 ('friday', 1.0): 91}

In [ ]:
# Exemplo de valores negativos            # Nesse trecho[6995 - 7000]:
dict(list(freqs.items())[6995:7000])      # Simpson - Apareceu 1 vez em tweets negativos
                                          # Concert - Apareceu 9 vezes em tweets negativos

{('ticket', 0.0): 9,
 ('codi', 0.0): 1,
 ('simpson', 0.0): 1,
 ('concert', 0.0): 9,
 ('singapor', 0.0): 3}

In [ ]:
# Todas as frequencias
freqs

{('followfriday', 1.0): 23,
 ('top', 1.0): 30,
 ('engag', 1.0): 7,
 ('member', 1.0): 14,
 ('commun', 1.0): 27,
 ('week', 1.0): 72,
 (':)', 1.0): 2847,
 ('hey', 1.0): 60,
 ('jame', 1.0): 7,
 ('odd', 1.0): 2,
 (':/', 1.0): 5,
 ('pleas', 1.0): 80,
 ('call', 1.0): 27,
 ('contact', 1.0): 4,
 ('centr', 1.0): 1,
 ('02392441234', 1.0): 1,
 ('abl', 1.0): 6,
 ('assist', 1.0): 1,
 ('mani', 1.0): 28,
 ('thank', 1.0): 504,
 ('listen', 1.0): 14,
 ('last', 1.0): 39,
 ('night', 1.0): 55,
 ('bleed', 1.0): 2,
 ('amaz', 1.0): 41,
 ('track', 1.0): 5,
 ('scotland', 1.0): 2,
 ('congrat', 1.0): 15,
 ('yeaaah', 1.0): 1,
 ('yipppi', 1.0): 1,
 ('accnt', 1.0): 2,
 ('verifi', 1.0): 2,
 ('rqst', 1.0): 1,
 ('succeed', 1.0): 1,
 ('got', 1.0): 57,
 ('blue', 1.0): 8,
 ('tick', 1.0): 1,
 ('mark', 1.0): 1,
 ('fb', 1.0): 4,
 ('profil', 1.0): 2,
 ('15', 1.0): 4,
 ('day', 1.0): 187,
 ('one', 1.0): 90,
 ('irresist', 1.0): 2,
 ('flipkartfashionfriday', 1.0): 16,
 ('like', 1.0): 187,
 ('keep', 1.0): 55,
 ('love', 1.0): 336,
 

## Modelagem Preditiva

Usaremos o algoritmo de Regressão Logística para classificação dos tweets em positivos ou negativos.

Vamos construir cada etapa matemática desse algoritmo.

### Parte 1:  Matemática da Função Sigmóide

A função sigmóide é uma função de ativação comumente usada em redes neurais. A função sigmóide é definida como:

$$ h(z) = \frac{1}{1+\exp^{-z}} $$

Ela tem a forma de uma curva S, como mostrado abaixo:

![title](imagens/sigmoid.png)

A função sigmóide tem a seguintes propriedades:

O valor de saída da função sigmóide está sempre entre 0 e 1. Isso torna a função útil para problemas de classificação binária, pois pode ser interpretada como a probabilidade de um determinado exemplo pertencer à classe positiva.

A função sigmóide é derivável em todos os pontos, o que a torna útil para o treinamento de redes neurais.

A função sigmóide tem um gradiente muito pequeno para valores de entrada muito grandes ou muito pequenos. Isso pode causar problemas durante o treinamento da rede neural, pois pode levar ao "estouro do gradiente", um problema em que o gradiente fica muito grande e a rede neural deixa de aprender de maneira eficiente.

Apesar desses problemas, a função sigmóide ainda é usada em alguns casos, especialmente em problemas de classificação binária. No entanto, outras funções de ativação, como a ReLU e a tangente hiperbólica, são mais comumente usadas em redes neurais profundas devido ao seu desempenho melhor.

In [ ]:
# Criando a função sigmóide
def sigmoid(z):
    # Calcula o sigmóide de z
    h = 1 / (1 + np.exp(-z))
    return h

In [ ]:
# Testando a função
print('O valor para 1 é:', sigmoid(1))
print('O valor para 3,92 é:', sigmoid(3.92))

O valor para 1 é: 0.7310585786300049
O valor para 3,92 é: 0.9805449154318069


In [ ]:
# Testando a função
if (sigmoid(1) == 0.7310585786300049):
    print('CORRETO!')
else:
    print('INCORRETO!')

CORRETO!


In [ ]:
# Testando a função
if (sigmoid(3.92) == 0.91):
    print('CORRETO!')
else:
    print('INCORRETO!')

INCORRETO!


### Parte 2: Matemática da Regressão Logística com a Função Sigmóide

A regressão logística é um modelo de aprendizado supervisionado usado para problemas de classificação binária. Ela é baseada em uma função de ativação chamada função sigmóide, que é usada para estimar a probabilidade de um exemplo pertencer à classe positiva.

A equação básica da regressão logística é dada por:

p = 1 / (1 + e^(-Wx + b))

Onde p é a probabilidade de um exemplo pertencer à classe positiva, x é o vetor de características de entrada, W é a matriz de pesos e b é o viés.

A função sigmóide é usada para transformar a saída linear da regressão logística em uma probabilidade. Quando a saída é maior que 0,5, o modelo prediz que o exemplo pertence à classe positiva, enquanto que quando a saída é menor que 0,5, o modelo prediz que o exemplo pertence à classe negativa.

Para treinar o modelo, usamos uma função de custo chamada entropia cruzada, que mede o quão incorretas são as previsões do modelo. O objetivo é minimizar a entropia cruzada ajustando os pesos W e o viés b. Isso é geralmente feito usando a descida do gradiente, um método de otimização baseado em derivadas.

Uma vez treinado, o modelo pode ser usado para fazer previsões para novos exemplos, usando a equação p = 1 / (1 + e^(-Wx + b)). Se a probabilidade p for maior que 0,5, o modelo prediz que o exemplo pertence à classe positiva, caso contrário, o modelo prediz que o exemplo pertence à classe negativa.

A regressão logística usa uma regressão linear regular e aplica um sigmóide à saída da regressão linear.

Regressão:

$$z = \theta_0 x_0 + \theta_1 x_1 + \theta_2 x_2 + ... \theta_N x_N$$

Observe que valores $\theta$ são "pesos".

Regressão Logística:

$$z = \theta_0 x_0 + \theta_1 x_1 + \theta_2 x_2 + ... \theta_N x_N$$

$$ h(z) = \frac{1}{1+\exp^{-z}}$$

Vamos nos referir a 'z' como 'logits'.

### Parte 3: Matemática da Função de Custo

A função de custo usada para regressão logística é a média da perda de log em todos os exemplos de treinamento:

$$J(\theta) = -\frac{1}{m} \sum_{i=1}^m y^{(i)}\log (h(z(\theta)^{(i)})) + (1-y^{(i)})\log (1-h(z(\theta)^{(i)})) $$

* $m$ é o número de exemplos de treinamento.
* $y^{(i)}$ é o rótulo real do exemplo de treinamento 'i'.
* $h(z^{(i)})$ é a previsão do modelo para o exemplo de treinamento 'i'.

A função de perda para um único exemplo de treinamento é

$$ Loss = -1 \times \left( y^{(i)}\log (h(z(\theta)^{(i)})) + (1-y^{(i)})\log (1-h(z(\theta)^{(i)})) \right)$$

* Todos os valores de $h$ estão entre 0 e 1, então os logs serão negativos. Essa é a razão do fator de -1 aplicado à soma dos dois termos de perda.


* Observe que quando o modelo prevê 1 ($h(z(\theta)) = 1$) e o rótulo 'y' também é 1, a perda para esse exemplo de treinamento é 0.


* Da mesma forma, quando o modelo prevê 0 ($h(z(\theta)) = 0$) e o rótulo real também é 0, a perda para esse exemplo de treinamento é 0.


* No entanto, quando a previsão do modelo está próxima de 1 ($h(z(\theta)) = 0,9999$) e o rótulo é 0, o segundo termo da perda de log torna-se um grande número negativo, que é então multiplicado pelo total fator de -1 para convertê-lo em um valor de perda positivo. $-1 \times (1 - 0) \times log(1 - 0.9999) \approx 9.2$.

### Parte 4: Matemática da Atualização dos Pesos (Descida do Gradiente)

Para atualizar o vetor de peso $\theta$, aplicamos a descida do gradiente para melhorar iterativamente as previsões do modelo.

O gradiente da função de custo $J$ em relação a um dos pesos $\theta_j$ é:

$$\nabla_{\theta_j}J(\theta) = \frac{1}{m} \sum_{i=1}^m(h^{(i)}-y^{(i)})x^{(i)}_j $$

* 'i' é o índice em todos os exemplos de treinamento 'm'.
* 'j' é o índice do peso $\theta_j$, então $x^{(i)}_j$ é o recurso associado ao peso $\theta_j$

* Para atualizar o peso $\theta_j$, ajustamos subtraindo uma fração do gradiente determinado por $\alpha$:

$$\theta_j = \theta_j - \alpha \times \nabla_{\theta_j}J(\theta) $$

* A taxa de aprendizado $\alpha$ é um valor que escolhemos para controlar o tamanho de uma única atualização.

### Parte 5: Combinando Todas as Operações e Criando o Algoritmo

* O número de iterações 'num_iters" é o número de vezes que você usará todo o conjunto de treinamento (número de passadas de treinamento).


* Para cada iteração, você calculará a função de custo usando todos os exemplos de treinamento (existem 'm' exemplos de treinamento) e para todos os recursos.


* Em vez de atualizar um único peso $\theta_i$ de cada vez, podemos atualizar todos os pesos no vetor de coluna:

$$\mathbf{\theta} = \begin{pmatrix}
\theta_0
\\
\theta_1
\\
\theta_2
\\
\vdots
\\
\theta_n
\end{pmatrix}$$

* $\mathbf{\theta}$ tem dimensões (n+1, 1), onde 'n' é o número de recursos e há mais um elemento para o termo de viés $\theta_0$.


* Os 'logits', 'z', são calculados multiplicando a matriz de características 'x' pelo vetor de peso 'theta'. $z = \mathbf{x}\mathbf{\theta}$
    * $\mathbf{x}$ tem dimensões (m, n+1)
    * $\mathbf{\theta}$: tem dimensões (n+1, 1)
    * $\mathbf{z}$: tem dimensões (m, 1)


* A predição 'h', é calculada aplicando o sigmóide a cada elemento em 'z': $h(z) = sigmoid(z)$, e tem dimensões (m,1).


* A função de custo $J$ é calculada através do produto escalar dos vetores 'y' e 'log(h)'. Como 'y' e 'h' são vetores coluna (m,1), fazemos a transposta do vetor para a esquerda, de modo que a multiplicação da matriz de um vetor linha com o vetor coluna execute o produto escalar.

$$J = \frac{-1}{m} \times \left(\mathbf{y}^T \cdot log(\mathbf{h}) + \mathbf{(1-y)}^T \cdot log(\mathbf{1-h}) \right)$$


* A atualização de $\theta$ também é vetorizada. Como as dimensões de $\mathbf{x}$ são (m, n+1) e $\mathbf{h}$ e $\mathbf{y}$ são (m, 1), precisamos transpor o $ \mathbf{x}$ à esquerda para realizar a multiplicação de matrizes, o que resulta na resposta (n+1, 1) de que precisamos:

$$\mathbf{\theta} = \mathbf{\theta} - \frac{\alpha}{m} \times \left( \mathbf{x}^T \cdot \left( \mathbf{h-y} \right) \right)$$

In [ ]:
# Função para o algoritmo de regressão logística
# Theta: Pesos
# Alpha: Taxa de aprendizado
def algo_reg_log(x, y, theta, alpha, num_iters):

    # Obter 'm', o número de linhas na matriz x
    m = x.shape[0]

    for i in range(0, num_iters):

        # Obter z, o produto escalar de x e teta
        z = np.dot(x,theta)

        # Obter sigmoid de h
        h = sigmoid(z)

        # Calcula a função de custo
        # Note que podemos usar também np.array.transpose() ao invés de np.array.T
        # np.array.T apenas torna o código um pouco mais legível
        J = -1./m * (np.dot(y.T, np.log(h)) + np.dot((1-y).T, np.log(1-h)))

        # Atualiza os pesos theta
        theta = theta - (alpha/m) * np.dot(x.T,(h-y))

    J = float(J)

    return J, theta


In [ ]:
# Seed
np.random.seed(42)

In [ ]:
# A entrada X é um array 10 x 3 com 1 para o termo de viés
dados_X = np.append(np.ones((10, 1)), np.random.rand(10, 2) * 2000, axis = 1)

In [ ]:
# Labels Y são array 10 x 1
dados_Y = (np.random.rand(10, 1) > 0.35).astype(float)

In [ ]:
# Aplica a função
valor_J, valor_theta = algo_reg_log(dados_X, dados_Y, np.zeros((3, 1)), 1e-8, 700)

In [ ]:
print(f"\nCusto (erro) após o treinamento é {valor_J:.8f}")
print(f"\nO vetor de pesos resultante é {[round(t, 8) for t in np.squeeze(valor_theta)]}")


Custo (erro) após o treinamento é 0.46889301

O vetor de pesos resultante é [6e-08, -0.00068256, 0.00097]


## Extração de Atributos

Com o algoritmo pronto vamos extrair os atributos dos dados e treinar um modelo.

* Dada uma lista de tweets, extraímos os recursos e armazenamos em um vetor. Vamos extrair dois recursos:

     * A primeira característica é o número de palavras positivas em um tweet.
     * A segunda característica é o número de palavras negativas em um tweet.


A função abaixo realiza as seguintes tarefas:

* Processa o tweet usando a função `limpa_processa_tweet` e salvamos a lista de palavras do tweet.


* Percorre cada palavra na lista de palavras processadas.


* Para cada palavra, verificamos o dicionário de frequências 'freqs' para a contagem quando essa palavra tiver um rótulo '1' positivo. Fazemos o mesmo para a contagem quando a palavra estiver associada ao rótulo negativo '0'.

In [ ]:
# Função para extração de atributos
def func_extract_features(tweet, freqs):

    # Aplica a função de limpeza e processamento
    palavra_l = limpa_processa_tweet(tweet)

    # Cria o vetor x de 3 elementos na forma 1 x 3
    x = np.zeros((1, 3))

    # O termo de bias será definido como 1
    x[0,0] = 1

    # Loop pelas palavras
    for palavra in palavra_l:

        # Busca a frequencia da palavra de tweet positivo
        x[0,1] += freqs.get((palavra, 1.0),0)

        # Busca a frequencia da palavra de tweet negativo
        x[0,2] += freqs.get((palavra, 0.0),0)

    # Valida o shape
    assert(x.shape == (1, 3))

    return x

## Treinamento do Modelo

Para treinar o modelo:

* Empilhamos os recursos de todos os exemplos de treinamento em uma matriz X.

* Executamos o algoritmo `algo_reg_log`, implementado anteriormente.

In [ ]:
# Criamos a matriz X
X = np.zeros((len(dados_treino_x), 3))

In [ ]:
# Loop para preencher a matriz com os dados
for i in range(len(dados_treino_x)):
    X[i, :]= func_extract_features(dados_treino_x[i], freqs)

In [ ]:
# Variável de saída (target)
Y = y_treino

In [ ]:
# Hiperparâmetros

# Valor inicial da matriz de pesos
matriz_pesos = np.zeros((3, 1))

# Taxa de aprendizado
taxa_aprendizado_alfa = 1e-9

# Número de iterações
num_iters = 1500

In [ ]:
# Treinamento do modelo
custo, pesos = algo_reg_log(X, Y, matriz_pesos, taxa_aprendizado_alfa, num_iters)

In [ ]:
print(f"O Custo (Erro) de Treinamento foi {custo:.8f}.")

O Custo (Erro) de Treinamento foi 0.24215478.


In [ ]:
print(f"O Vetor de Pesos é {[round(t, 8) for t in np.squeeze(pesos)]}")

O Vetor de Pesos é [7e-08, 0.00052391, -0.00055517]


São 3 pesos pois temos 2 atributos de entrada e o bias.

## Previsões com o Modelo

É hora de fazer previsões com o modelo de regressão logística em alguma nova entrada de dados. O objetivo é prever se um tweet tem sentimento positivo ou negativo.

Vamos criar uma função para isso que executará as seguintes tarefas:

* Dado um tweet, processamos e extraímos os recursos (o que mesmo que foi feito nso dados de treino).
* Aplicamos os pesos aprendidos do modelo para obter os logits.
* Aplicamos a função sigmóide aos logits para obter a previsão (um valor entre 0 e 1).

Resumindo:

$$y_{pred} = sigmoide(\mathbf{x} \cdot \theta)$$

In [ ]:
# Função para previsão
def func_previsao(tweet, freqs, pesos):

    # Extrai os atributos
    x = func_extract_features(tweet, freqs)

    # Faz a previsao
    y_pred = sigmoid(np.dot(x, pesos))

    return y_pred

In [ ]:
# Vamos testar a função
for tweet in [':)',
              ':(',
              'I am happy',
              'This course is great',
              'I do not expect so much from my soccer team',
              'It was a good book',
              'I am not sure about the text']:
    print( '%s -> %f' % (tweet, func_previsao(tweet, freqs, pesos)))

:) -> 0.816147
:( -> 0.115773
I am happy -> 0.518581
This course is great -> 0.515933
I do not expect so much from my soccer team -> 0.494500
It was a good book -> 0.513107
I am not sure about the text -> 0.501005


## Avaliação do Modelo

Vamos agora avaliar a performance do modelo.

Vamos criar uma função para testar o modelo que executará as seguintes tarefas:

* Recebe os dados de teste e os pesos do modelo treinado e calcula a precisão do modelo.
* Usa a função `func_previsao` para fazer previsões sobre cada tweet no conjunto de teste.
* Se a previsão for > 0,5, definimos a classificação do modelo 'y_hat' como 1, caso contrário, definimos a classificação do modelo `y_pred` como 0.
* Uma previsão é precisa quando o `y_pred` é igual ao `y_teste`.

In [ ]:
# Função para testar o modelo
def func_testa_modelo(test_x, test_y, freqs, theta):

    # Lista para s previsões
    y_hat = []

    # Loop pelos dados
    for tweet in test_x:

        # Faz a previsão
        y_pred = func_previsao(tweet, freqs, theta)

        # Cutoff
        if y_pred > 0.5:
            y_hat.append(1)
        else:
            # append 0 to the list
            y_hat.append(0)

    # Calcula a acurácia
    accuracy = (y_hat==np.squeeze(test_y)).sum() / len(test_x)

    return accuracy

In [ ]:
acuracia = func_testa_modelo(dados_teste_x, y_teste, freqs, pesos)

In [ ]:
print(f"Acurácia do Modelo = {acuracia:.4f}")

Acurácia do Modelo = 0.9950


## Deploy do Modelo Treinado e Uso com Novos Dados

In [ ]:
# Cria um tweet
meu_tweet_1 = 'This is a great course. I am learning a lot!'

In [ ]:
print(limpa_processa_tweet(meu_tweet_1))

['great', 'cours', 'learn', 'lot']


In [ ]:
# Previsão
y_hat = func_previsao(meu_tweet_1, freqs, pesos)
print(y_hat)

[[0.5242686]]


In [ ]:
# Cutoff
if y_hat > 0.5:
    print('O Tweet tem sentimento positivo!')
else:
    print('O Tweet tem sentimento negativo!')

O Tweet tem sentimento positivo!


## Inserção manual

In [ ]:
# Cria um tweet
meu_tweet_aleatorio = input('Insira o seu tweet em inglês: ')

# Limpa o tweet
print(limpa_processa_tweet(meu_tweet_aleatorio))

# Previsão
y_hat = func_previsao(meu_tweet_aleatorio, freqs, pesos)
print(y_hat)

# Cutoff
if y_hat > 0.5:
    print('O Tweet tem um sentimento positivo maior!')
else:
    print('O Tweet tem um sentimento negativo menor!')

Insira o seu tweet em inglês: I hate this
['hate']
[[0.49493327]]
O Tweet tem um sentimento negativo menor!
